<a href="https://colab.research.google.com/github/shubhamjha16/jira_bug_restapi_project/blob/master/jira_pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required libraries

In [7]:
!pip install requests pandas

2. Setup Jira Credentials

In [8]:
import os
from google.colab import userdata

API_TOKEN = userdata.get("JIRA_TOKEN")
EMAIL = userdata.get("JIRA_EMAIL")
JIRA_URL = userdata.get("JIRA_URL")
PROJECT_KEY = userdata.get("PROJECT_KEY")

 3. Define the Bug List

In [9]:
bugs = [
    {
        "summary": "Login form accepts empty password",
        "description": "Steps:\n1. Go to login\n2. Leave password empty\n3. Submit\nExpected: error\nActual: login"
    },
    {
        "summary": "Dark mode not applied on profile page",
        "description": "Steps:\n1. Enable dark mode\n2. Go to profile\nExpected: dark theme\nActual: light"
    },
    {
        "summary": "Search results don't update after deleting a filter",
        "description": "Steps:\n1. Search with filters\n2. Remove filter\nExpected: refreshed results\nActual: same results"
    },
    {
        "summary": "Error 500 when uploading a 10MB file",
        "description": "Steps:\n1. Upload 10MB file\n2. Submit\nExpected: success or friendly error\nActual: 500 server error"
    },
    {
        "summary": "Notification badge count doesn't reset after reading messages",
        "description": "Steps:\n1. Open new messages\nExpected: badge count = 0\nActual: still shows count"
    }
]

 Create Bugs in Jira

In [10]:
import requests
import json
from requests.auth import HTTPBasicAuth

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

url = f"{JIRA_URL}/rest/api/3/issue"

for bug in bugs:
    # Format description in Atlassian Document Format (ADF)
    description_adf = {
        "type": "doc",
        "version": 1,
        "content": [
            {
                "type": "paragraph",
                "content": [
                    {
                        "type": "text",
                        "text": bug["description"]
                    }
                ]
            }
        ]
    }

    payload = json.dumps({
        "fields": {
            "project": {"key": PROJECT_KEY},
            "summary": bug["summary"],
            "description": description_adf,
            "issuetype": {"name": "Bug"}
        }
    })

    response = requests.post(
        url,
        headers=headers,
        data=payload,
        auth=HTTPBasicAuth(EMAIL, API_TOKEN)
    )

    if response.status_code == 201:
        print("✅ Created:", response.json()["key"])
    else:
        print("❌ Error:", response.status_code)
        print(response.text)

✅ Created: ECS-17
✅ Created: ECS-18
✅ Created: ECS-19
✅ Created: ECS-20
✅ Created: ECS-21


Fetch All Bugs from Jira

In [11]:
import pandas as pd

search_url = f"{JIRA_URL}/rest/api/3/search"
params = {
    "jql": f"project = {PROJECT_KEY} AND issuetype = Bug ORDER BY created DESC",
    "fields": "summary,description,created,status"
}

response = requests.get(
    search_url,
    headers={"Accept": "application/json"},
    params=params,
    auth=HTTPBasicAuth(EMAIL, API_TOKEN)
)

if response.status_code == 200:
    issues = response.json()["issues"]
    bugs_data = [{
        "Key": i["key"],
        "Summary": i["fields"]["summary"],
        "Description": i["fields"].get("description", ""),
        "Created": i["fields"]["created"],
        "Status": i["fields"]["status"]["name"]
    } for i in issues]

    df = pd.DataFrame(bugs_data)
    df.to_csv("jira_bugs.csv", index=False)
    print("✅ Saved to jira_bugs.csv")
    df.head()
else:
    print("❌ Error:", response.status_code)
    print(response.text)

✅ Saved to jira_bugs.csv


Download the CSV

In [12]:
from google.colab import files
files.download("jira_bugs.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Install Plotly

In [13]:
!pip install plotly

Load the CSV

In [14]:
import pandas as pd
import plotly.express as px

# Load your bug data
df = pd.read_csv("jira_bugs.csv")

# Convert 'Created' to datetime
df["Created"] = pd.to_datetime(df["Created"])
df["Created_Date"] = df["Created"].dt.date

Bug Count by Status

In [15]:
fig1 = px.histogram(
    df,
    x="Status",
    color="Status",
    title="🪲 Bug Count by Status",
    text_auto=True
)
fig1.show()

Bugs Over Time

In [16]:
fig2 = px.histogram(
    df,
    x="Created_Date",
    title="📆 Bugs Created Over Time",
    nbins=20
)
fig2.update_xaxes(title="Date")
fig2.update_yaxes(title="Number of Bugs")
fig2.show()

Top Keywords in Bug Summary (Word Count)

In [17]:
from collections import Counter
import re

# Basic word frequency in summaries
words = " ".join(df["Summary"]).lower()
words = re.findall(r'\b\w+\b', words)
word_freq = Counter(words)

top_words = pd.DataFrame(word_freq.most_common(10), columns=["Word", "Count"])

fig3 = px.bar(
    top_words,
    x="Word",
    y="Count",
    title="🔍 Most Frequent Words in Bug Summaries",
    text_auto=True
)
fig3.show()

 Interactive Dropdown Filter

In [18]:
import ipywidgets as widgets
from IPython.display import display

status_options = sorted(df["Status"].unique())

dropdown = widgets.Dropdown(
    options=status_options,
    description="Filter by Status:"
)

def filter_table(status):
    display(df[df["Status"] == status][["Key", "Summary", "Created", "Status"]])

widgets.interact(filter_table, status=dropdown)

interactive(children=(Dropdown(description='Filter by Status:', options=('To Do',), value='To Do'), Output()),…

<function __main__.filter_table(status)>

LOADING OF CSV FOR DATABASE

In [21]:
import pandas as pd
import plotly.express as px
df = pd.read_csv("jira_bugs.csv")
df["Created"] = pd.to_datetime(df["Created"])
df["Created_Date"] = df["Created"].dt.date

INSERTION OF BUGS INTO SUPABASE DATA TABLE jira.pilot_bug

In [22]:
import requests
from google.colab import userdata


SUPABASE_URL = userdata.get("SUPABASE_URL")
SUPABASE_KEY = userdata.get("SUPABASE_KEY")
TABLE_NAME = userdata.get("TABLE_NAME")


headers = {
    "apikey": SUPABASE_KEY,
    "Authorization": f"Bearer {SUPABASE_KEY}",
    "Content-Type": "application/json"
}


for _, row in df.iterrows():
    bug = {
        "key": row["Key"],
        "summary": row["Summary"],
        "description": row["Description"],
        "status": row["Status"],
        "created": row["Created"].isoformat()
    }

    response = requests.post(
        f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}",
        headers=headers,
        json=bug
    )

    if response.status_code in [200, 201]:
        print(f"✅ Inserted: {bug['key']}")
    else:
        print(f"❌ Error inserting {bug['key']}:", response.status_code)
        print(response.text)

✅ Inserted: ECS-21
✅ Inserted: ECS-20
✅ Inserted: ECS-19
✅ Inserted: ECS-18
✅ Inserted: ECS-17
✅ Inserted: ECS-16
✅ Inserted: ECS-15
✅ Inserted: ECS-14
✅ Inserted: ECS-13
✅ Inserted: ECS-12
✅ Inserted: ECS-11
✅ Inserted: ECS-10
✅ Inserted: ECS-9
✅ Inserted: ECS-8
✅ Inserted: ECS-7


In [23]:
code = """
import streamlit as st
import pandas as pd
import requests
import plotly.express as px
import os

# Set up page
st.set_page_config(page_title="Real-Time Jira Bug Dashboard", layout="wide")
st.title("🐞 Real-Time Jira Bug Dashboard (from Supabase)")

# Supabase info
SUPABASE_URL = "https://npwzddejukdbaxgasrsj.supabase.co"
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")  # 🔐 Secure via Streamlit Secrets
TABLE_NAME = "jira.pilot_bug"

@st.cache_data(ttl=60)
def fetch_bugs():
    headers = {
        "apikey": SUPABASE_KEY,
        "Authorization": f"Bearer {SUPABASE_KEY}"
    }
    url = f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}?select=*"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        st.error("Failed to fetch bugs from Supabase")
        return pd.DataFrame()
    return pd.DataFrame(response.json())

df = fetch_bugs()

if df.empty:
    st.stop()

df["created"] = pd.to_datetime(df["created"])
df["created_date"] = df["created"].dt.date

with st.sidebar:
    st.header("🔍 Filters")
    status_filter = st.multiselect("Status", df["status"].unique(), default=list(df["status"].unique()))
    date_range = st.date_input("Created Date Range", [df["created_date"].min(), df["created_date"].max()])

filtered_df = df[
    (df["status"].isin(status_filter)) &
    (df["created_date"] >= date_range[0]) &
    (df["created_date"] <= date_range[1])
]

col1, col2 = st.columns(2)

with col1:
    bar_df = filtered_df.groupby("created_date").size().reset_index(name="Bug Count")
    fig1 = px.bar(bar_df, x="created_date", y="Bug Count", title="📅 Bugs Created Per Day")
    st.plotly_chart(fig1, use_container_width=True)

with col2:
    fig2 = px.pie(filtered_df, names="status", title="📊 Bug Status Distribution")
    st.plotly_chart(fig2, use_container_width=True)

st.subheader("📋 Bug List")
st.dataframe(filtered_df[["key", "summary", "status", "created"]].sort_values("created", ascending=False), use_container_width=True)
"""

In [24]:
with open("app.py", "w") as f:
    f.write(code)

In [25]:
from google.colab import files
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>